# 7 数据清洗与准备
重点讨论缺失值、重复值、字符串操作和其他数据转换工具

## 7.1 处理缺失值

In [1]:
# pandas对象的所有描述性统计信息默认情况下是排除缺失值的
import pandas as pd
import numpy as np

string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [2]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
# NA意思是not available（不可用）
# 当数据清洗时，对缺失数据本身进行分析已确定数据收集问题或数据丢失导致的数据偏差通常很重要。
# None在对象数组中也被当作NA处理
string_data[0] = None
string_data

0         None
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 7.1.1 过滤缺失值

In [6]:
from numpy import nan as NA

data = pd.Series([NA, 1, 2, NA, 4])
# Series中使用dropna，会返回所有非NA的数据
data.dropna()

1    1.0
2    2.0
4    4.0
dtype: float64

In [8]:
data[data.notnull()]

1    1.0
2    2.0
4    4.0
dtype: float64

In [11]:
# dropna默认删除包含NA的行
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.55, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.55,3.0


In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [13]:
# 当传入how='all'时，会删除所有值均为NA的行
data.dropna(how='all')

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
3,NaN,6.55,3.0


In [15]:
data[4] = NA

In [16]:
data

,0,1,2,4
0,1.0,6.50,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.55,3.0,NaN


In [18]:
# 删除列加上axis=1
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.50,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.55,3.0


In [19]:
# 只想保留一定数量的行，使用thresh参数来表示
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,2.220267,NaN,NaN
1,1.648893,NaN,NaN
2,1.502815,NaN,-1.576268
3,-0.791778,NaN,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [20]:
df.dropna()

,0,1,2
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [21]:
df.dropna(thresh=2)

,0,1,2
2,1.502815,NaN,-1.576268
3,-0.791778,NaN,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


### 7.1.2 补全缺失值

In [23]:
# 大多数情况下使用fillna来不全缺失值
df.fillna(0)

,0,1,2
0,2.220267,0.000000,0.000000
1,1.648893,0.000000,0.000000
2,1.502815,0.000000,-1.576268
3,-0.791778,0.000000,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [24]:
# 可以使用字典来为不同的列填充不同的值
df.fillna({1: 0.5, 2: 1})

,0,1,2
0,2.220267,0.500000,1.000000
1,1.648893,0.500000,1.000000
2,1.502815,0.500000,-1.576268
3,-0.791778,0.500000,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [27]:
# fillna默认不会修改原有的对象，加inplace=True会就地修改
_ = df.fillna({1: 0.5, 2: 1}, inplace=True)
df

,0,1,2
0,2.220267,0.500000,1.000000
1,1.648893,0.500000,1.000000
2,1.502815,0.500000,-1.576268
3,-0.791778,0.500000,-0.098838
4,0.206316,0.207775,0.431345
5,0.552498,-1.905871,0.377752
6,-0.794606,-2.397310,-1.736436


In [28]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.365008,0.284544,-0.677766
1,0.592913,-1.810273,-2.835809
2,0.256803,NaN,-0.688223
3,-1.758211,NaN,1.070885
4,-0.309773,NaN,NaN
5,-0.772189,NaN,NaN


In [30]:
# 使用前一项值来填充空值
df.fillna(method='ffill')

,0,1,2
0,0.365008,0.284544,-0.677766
1,0.592913,-1.810273,-2.835809
2,0.256803,-1.810273,-0.688223
3,-1.758211,-1.810273,1.070885
4,-0.309773,-1.810273,1.070885
5,-0.772189,-1.810273,1.070885


In [31]:
# 也可以使用均值等来填充缺失值
df.fillna(df.mean())

,0,1,2
0,0.365008,0.284544,-0.677766
1,0.592913,-1.810273,-2.835809
2,0.256803,-0.762864,-0.688223
3,-1.758211,-0.762864,1.070885
4,-0.309773,-0.762864,-0.782728
5,-0.772189,-0.762864,-0.782728


## 7.2 数据转换

### 7.2.1 删除重复值

In [32]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [33]:
# 返回行是否重复
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [34]:
# 删除重复的行
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [35]:
# duplicated和drop_duplicates默认都是保留地一个观测到的值
# 传入keep='last'就会保留最后一个
data.drop_duplicates(keep='last')

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
6,two,4


### 7.2.2 使用函数或Map进行数据转换

In [36]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [37]:
# 想要添加一列用于表明每种食物的动物肉类型，先写一个Map
meat_to_animal = {'bacon': 'pig',
                  'pulled pork': 'pig',
                  'pastrami': 'cow',
                  'corned beef': 'cow',
                  'honey ham': 'pig',
                  'nova lox': 'salmon'}

In [39]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [40]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [45]:
# 使用匿名函数，来完成转换工作
data['animal1'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal,animal1
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,Pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,Bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


### 7.2.3 替代值

In [46]:
# 使用replace
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [47]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [48]:
# 一次性替代多个值
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [49]:
# 将不同的值替换为不同的值
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.2.3 重命名轴索引

In [51]:
data = pd.DataFrame(np.arange(12).reshape(3, 4),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [54]:
# 转换df的轴索引
transform = lambda x: x[:4].upper()
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [55]:
# 如果想要创建数据集转换后的版本，并且不修改原有的数据集，一个有用的方法时rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [58]:
# 想要修改原有数据集
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 离散化和分箱

In [60]:
ages = [20, 22, 25, 27, 21, 23, 27, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cate = pd.cut(ages, bins)
cate

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 13
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [61]:
# 返回分箱名的字符串数组
cate.codes

array([0, 0, 0, 1, 0, 0, 1, 2, 1, 3, 2, 2, 1], dtype=int8)

In [62]:
cate.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [65]:
# 统计各区间的数量
pd.value_counts(cate)

(18, 25]     5
(25, 35]     4
(35, 60]     3
(60, 100]    1
dtype: int64

In [66]:
# 使用right=False来改变哪一边是封闭的
pd.cut(ages, [18, 25, 35, 60, 100], right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 13
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [67]:
# 可以自定义箱名
group_names = ['Youth', 'YouthAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YouthAdult', 'Youth', ..., 'YouthAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YouthAdult']
Length: 13
Categories (4, object): ['Youth' < 'YouthAdult' < 'MiddleAged' < 'Senior']

In [70]:
# 如果传递的bins是整数，pandas将根据数据中的最小值和最大值计算等长的箱
data = np.random.randn(20)
pd.cut(data, 4, precision=2)

[(-0.41, 0.74], (-1.56, -0.41], (-0.41, 0.74], (-0.41, 0.74], (-1.56, -0.41], ..., (-0.41, 0.74], (0.74, 1.89], (-0.41, 0.74], (-0.41, 0.74], (-1.56, -0.41]]
Length: 20
Categories (4, interval[float64, right]): [(-2.72, -1.56] < (-1.56, -0.41] < (-0.41, 0.74] < (0.74, 1.89]]

In [73]:
# qcut是根据分位数进行分箱，取决于数据的分布，
# 使用cut通常不会获得相同长度的数据量的数据点
# qcut使用样本的分位数，可以获得等长的分箱
data = np.random.randn(1000)
cats = pd.qcut(data, 4)  # 切成4份
cats

[(-0.748, 0.000507], (0.656, 2.934], (0.000507, 0.656], (0.656, 2.934], (-0.748, 0.000507], ..., (-2.767, -0.748], (-2.767, -0.748], (0.000507, 0.656], (0.000507, 0.656], (0.000507, 0.656]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.767, -0.748] < (-0.748, 0.000507] < (0.000507, 0.656] < (0.656, 2.934]]

In [74]:
pd.value_counts(cats)

(-2.767, -0.748]      250
(-0.748, 0.000507]    250
(0.000507, 0.656]     250
(0.656, 2.934]        250
dtype: int64

In [77]:
# 与cut类似，可以传入自定义个分位数
cats1 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])

In [78]:
pd.value_counts(cats1)

(-1.327, 0.000507]    400
(0.000507, 1.243]     400
(-2.767, -1.327]      100
(1.243, 2.934]        100
dtype: int64

### 7.2.6 检测和过滤异常值

In [79]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.015005,-0.001289,-0.006958,0.030142
std,1.011158,0.961967,1.000396,1.019316
min,-3.013471,-3.458879,-3.478172,-3.548921
25%,-0.666373,-0.651261,-0.675810,-0.645341
50%,0.005520,0.037229,-0.008796,0.040111
75%,0.666534,0.630404,0.641830,0.698611
max,3.078613,2.771729,2.997883,3.789551


In [80]:
# 找出绝对值大于3的值
col = data[2]
col[np.abs(col) > 3]

616   -3.478172
Name: 2, dtype: float64

In [81]:
# 找出绝对值大于3的所有行
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
106,3.078613,0.438715,1.111370,-0.328266
343,1.020538,0.928505,0.563089,-3.548921
616,-0.890646,-0.949517,-3.478172,-0.531201
619,3.024883,-0.277389,1.441836,-1.388401
716,-0.686979,-3.458879,0.409756,-0.815935
810,-3.003978,-0.671287,-0.536794,-0.348704
911,-3.013471,0.790675,-0.129927,0.407799
995,-2.150107,-0.655749,-0.722745,3.789551


In [82]:
# 将绝对值大于3的正负数分别转换为3和-3
# np.sign是将正数转换为1，负数转换为-1
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.015091,-0.000830,-0.006480,0.029901
std,1.010794,0.960424,0.998848,1.014916
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.666373,-0.651261,-0.675810,-0.645341
50%,0.005520,0.037229,-0.008796,0.040111
75%,0.666534,0.630404,0.641830,0.698611
max,3.000000,2.771729,2.997883,3.000000


### 7.2.7 置换和随机抽样

In [87]:
df = pd.DataFrame(np.arange(5 * 4).reshape(5, 4))
# np.random.permutation生成一个随机序列
sampler = np.random.permutation(5)
type(sampler)

numpy.ndarray

In [88]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [89]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19


In [90]:
# 返回一个随机子集
df.sample(3)

,0,1,2,3
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15


In [91]:
# 要生成一个带有替代值的样本（允许有重复选择）
choice = pd.Series([5, -7, -1, 6, 4])
choice.sample(n=10, replace=True)

0    5
2   -1
4    4
1   -7
1   -7
3    6
3    6
1   -7
3    6
1   -7
dtype: int64

### 7.2.8 计算指标和虚拟变量

In [92]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [93]:
# 根据一列内容k个不同的值，可以衍生出k列的值为0或1的矩阵或DataFrame
# 对应位置上有值则为1，否则为0
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [94]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [95]:
# 如果一行属于多个类别，则比较复杂，如电影的分类
movie_name = ['movie_id', 'title', 'genres']
movies = pd.read_table("./examples/movies.dat", sep="::", header=None, names=movie_name)
movies[:10]

/var/folders/ml/8mffbtvx25b6ftr05mlfbtqm0000gn/T/ipykernel_17823/981819977.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table("./examples/movies.dat", sep="::", header=None, names=movie_name)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [97]:
# 首先提取出电影的流派
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres.tolist()

['Animation',
 "Children's",
 'Comedy',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Sci-Fi',
 'Documentary',
 'War',
 'Musical',
 'Mystery',
 'Film-Noir',
 'Western']

In [102]:
# 首先构建一个全0的DataFrame
zero_matrix = np.zeros((len(movies), len(genres)))
# 生成一个DataFrame
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head(5)

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
# 遍历movies，将dummies中符合条件的条目置为1
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))  # 获取值对应列名的索引
    dummies.iloc[i, indices] = 1

In [104]:
dummies.head(10)

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                              1.0
Genre_Children's                             1.0
Genre_Comedy                                 1.0
Genre_Adventure                              0.0
Genre_Fantasy                                0.0
Genre_Romance                                0.0
Genre_Drama                                  0.0
Genre_Action                                 0.0
Genre_Crime                                  0.0
Genre_Thriller                               0.0
Genre_Horror                                 0.0
Genre_Sci-Fi                                 0.0
Genre_Documentary                            0.0
Genre_War                                    0.0
Genre_Musical                                0.0
Genre_Mystery                                0.0
Genre_Film-Noir                              0.0
Genre_Western       

In [114]:
# 将get_dummies与cut结合使用
data = np.random.randn(10)
df = pd.DataFrame(data)
bins = [-np.inf, 0, 0.2, 0.4, 0.6, 0.8, 1, np.inf]
dummies_1 = pd.get_dummies(pd.cut(data, bins))

In [115]:
df.join(dummies_1)

,0,"(-inf, 0.0]","(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]","(1.0, inf]"
0,-0.218351,1,0,0,0,0,0,0
1,-1.029741,1,0,0,0,0,0,0
2,1.776850,0,0,0,0,0,0,1
3,-0.211946,1,0,0,0,0,0,0
4,-0.199138,1,0,0,0,0,0,0
5,-0.111205,1,0,0,0,0,0,0
6,1.259349,0,0,0,0,0,0,1
7,0.007392,0,1,0,0,0,0,0
8,-1.730749,1,0,0,0,0,0,0
9,-0.710779,1,0,0,0,0,0,0
